In [ ]:
from pyecharts import Graph

nodes = [{"name": n, "symbolSize": info[info.link_ID == n]['width'].values[0]} for n in adj.link_ID.values]

links = []
for (link, inl, outl) in adj.values:
    for ol in outl.split("#"):
        links.append({"source": link, "target": ol, "value": info[info.link_ID==link]['length'].values[0]+(0 if ol == '' else info[info.link_ID==ol]['length'].values[0])})
    for il in inl.split("#"):
        links.append({"source": il, "target": link, "value": info[info.link_ID==link]['length'].values[0]+(0 if il == '' else info[info.link_ID==il]['length'].values[0])})
    
graph = Graph("道路图", width=1600, height=800)
graph.add("", nodes, links, is_label_show=False, repulsion=1000, label_text_color=None, gravity=0.001)
# graph.show_config()
graph.render()

In [3]:
import h5py
f = h5py.File('travel_time_second.hdf5', 'r')
links = f.keys()

In [ ]:
import pandas as pd
import numpy as np
h5 = f[links[10]]['train']
train = pd.DataFrame(np.hstack((h5['data'].value, h5['time'].value.reshape(-1, 1))),
            columns=['travel_time', 'fill', 'hour', 'time'])
train['time'] = pd.to_datetime(train.time)
train.head()

In [ ]:
train['travel_time'] = train.travel_time.astype(np.float32)
train['fill'] = train.fill.astype(np.float32)
train.index = train.time

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

train['travel_time'] = scaler.fit_transform(train.travel_time)

In [ ]:
%matplotlib inline

In [ ]:
for day in ['0514', '0521', '0516', '0523']:
    train[('2016'+day+'0600'):('2016'+day+'0900')].plot(y=['travel_time', 'fill'], title=day)